In [ ]:
import torch
import numpy as np

# 1.1) Tensors and Operations

### **Q 1.1 a)**

In [ ]:
shape = (3,2)
x = torch.rand(shape)
print(x)

tensor([[0.9334, 0.1717],
        [0.7645, 0.7880],
        [0.8824, 0.7022]])


### **Q 1.1 b)**

In [ ]:
y = torch.ones(shape)
print(y)
print(y.shape)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
torch.Size([3, 2])


### **Q 1.1 c)**

In [ ]:
out = torch.add(x,y)
print(out)

tensor([[1.9334, 1.1717],
        [1.7645, 1.7880],
        [1.8824, 1.7022]])


In [ ]:
y.add_(x)
print(y)

tensor([[1.9334, 1.1717],
        [1.7645, 1.7880],
        [1.8824, 1.7022]])


### **Q 1.1 d)**

In [ ]:
x = np.random.rand(3,2)
x = torch.from_numpy(x)
print(x)
x = x.numpy()
print(x)

tensor([[0.9910, 0.2627],
        [0.2259, 0.5102],
        [0.4148, 0.0354]], dtype=torch.float64)
[[0.99104231 0.26271163]
 [0.2259091  0.51024524]
 [0.41481212 0.03542314]]


#1.2) Autograd

### **Q 1.2 a)**

In [ ]:
shape = (3,2)
x = torch.rand(shape, requires_grad=True)
print(x)

tensor([[0.0720, 0.9404],
        [0.2588, 0.5108],
        [0.7300, 0.6649]], requires_grad=True)


### **Q 1.2 b)**

In [ ]:
y = x.mul(10)
print("Multiplied by 10: " + str(y))
print("\n")
y.add_(0.1)
print("Addition by 0.1: " + str(y))
print("\n")
out = torch.max(y)
print("Out: " + str(out))


Multiplied by 10: tensor([[0.7200, 9.4043],
        [2.5880, 5.1084],
        [7.3003, 6.6487]], grad_fn=<MulBackward0>)


Addition by 0.1: tensor([[0.8200, 9.5043],
        [2.6880, 5.2084],
        [7.4003, 6.7487]], grad_fn=<AddBackward0>)


Out: tensor(9.5043, grad_fn=<MaxBackward1>)


### **Q 1.2 c)**

In [ ]:
out.backward()
print(x.grad)

tensor([[ 0., 10.],
        [ 0.,  0.],
        [ 0.,  0.]])


### **Q 1.2 d)**

In [ ]:
with torch.no_grad():
  y = x.mul(10)
  print("Multiplied by 10: " + str(y))
  print("\n")
  y.add_(0.1)
  print("Addition by 0.1: " + str(y))
  print("\n")
  out = torch.max(y)
  print("Out: " + str(out))

Multiplied by 10: tensor([[0.7200, 9.4043],
        [2.5880, 5.1084],
        [7.3003, 6.6487]])


Addition by 0.1: tensor([[0.8200, 9.5043],
        [2.6880, 5.2084],
        [7.4003, 6.7487]])


Out: tensor(9.5043)


# 1.3) Neural Networks


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3,8,5)
    self.conv2 = nn.Conv2d(8,24,3)

    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)

    self.fc1 = nn.Linear(24*6*6, 240)
    self.fc2 = nn.Linear(240, 120)
    self.fc3 = nn.Linear(120, 84)
    self.fc4 = nn.Linear(84, 10)
    
  
  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)

    x = x.view(-1, self.num_flat_features(x))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)

    # x = F.log_softmax(x, dim=1)

    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
        num_features *= s
    return num_features


### **Q 1.3 a)**

In [ ]:
net = Net()
print("Network : " + str(net))
print("\n")
params = list(net.parameters())
print(params[0].size())

Network : Net(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(8, 24, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=864, out_features=240, bias=True)
  (fc2): Linear(in_features=240, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=84, bias=True)
  (fc4): Linear(in_features=84, out_features=10, bias=True)
)


torch.Size([8, 3, 5, 5])


### **Q 1.3 b)**

In [ ]:
input = torch.randn(1, 3, 32, 32)
out = net(input)
print(out)
print("\n")
shape_out = tuple(out.size())
print(shape_out)

tensor([[ 0.1027,  0.0314, -0.0525, -0.0677,  0.0400, -0.0721,  0.0813,  0.0695,
         -0.0231,  0.0849]], grad_fn=<AddmmBackward>)


(1, 10)


### **Q 1.3 c)**

In [ ]:
target_out = torch.rand(shape_out)
print(target_out)
print("\n")

loss = nn.MSELoss()
output = loss(out, target_out)

optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output.backward()
optimizer.step()


tensor([[0.5779, 0.3176, 0.1758, 0.9342, 0.8531, 0.5869, 0.8852, 0.5225, 0.9552,
         0.3400]])




In [ ]:
print('conv2.bias.grad after backward')
print(net.conv2.bias.grad)

conv2.bias.grad after backward
tensor([ 0.0023, -0.0004,  0.0027, -0.0012,  0.0063, -0.0070,  0.0008, -0.0017,
         0.0032,  0.0008,  0.0032,  0.0000,  0.0015, -0.0003, -0.0013, -0.0002,
         0.0030,  0.0011, -0.0017, -0.0033,  0.0016,  0.0041, -0.0016,  0.0059])


# 1.4) Training



In [ ]:
rm -rf ./data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.optim as optim
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      # print statistics
      running_loss += loss.item()
      if i % 2000 == 1999:
          print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
          running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.262
[1,  4000] loss: 2.023
[1,  6000] loss: 1.784
[1,  8000] loss: 1.610
[1, 10000] loss: 1.498
[1, 12000] loss: 1.442
[2,  2000] loss: 1.355
[2,  4000] loss: 1.331
[2,  6000] loss: 1.288
[2,  8000] loss: 1.277
[2, 10000] loss: 1.224
[2, 12000] loss: 1.223
[3,  2000] loss: 1.111
[3,  4000] loss: 1.112
[3,  6000] loss: 1.085
[3,  8000] loss: 1.084
[3, 10000] loss: 1.073
[3, 12000] loss: 1.074
Finished Training


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 68.3 %
Accuracy for class car   is: 68.1 %
Accuracy for class bird  is: 54.5 %
Accuracy for class cat   is: 33.8 %
Accuracy for class deer  is: 50.3 %
Accuracy for class dog   is: 53.2 %
Accuracy for class frog  is: 68.3 %
Accuracy for class horse is: 74.4 %
Accuracy for class ship  is: 81.9 %
Accuracy for class truck is: 74.6 %


# 1.5) Transfer Learning




In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        model.train()

        running_loss = 0.0
        running_corrects = 0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
                    
            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return model

### **Q 1.5 a)**

In [ ]:
from torchvision import datasets, models, transforms
import torch.optim as optim
import torch
import torch.nn as nn
import time


model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10, bias=True)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer, 3)

Epoch 0/2
----------
[1,  2000] loss: 2.732
[1,  4000] loss: 2.515
[1,  6000] loss: 2.468
[1,  8000] loss: 2.425
[1, 10000] loss: 2.381
[1, 12000] loss: 2.418
Epoch 1/2
----------
[2,  2000] loss: 2.309
[2,  4000] loss: 2.279
[2,  6000] loss: 2.189
[2,  8000] loss: 2.162
[2, 10000] loss: 2.148
[2, 12000] loss: 2.167
Epoch 2/2
----------
[3,  2000] loss: 2.087
[3,  4000] loss: 2.052
[3,  6000] loss: 2.019
[3,  8000] loss: 1.989
[3, 10000] loss: 1.956
[3, 12000] loss: 1.934
Training complete in 7m 33s


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 32 %


In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_ft(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 31.4 %
Accuracy for class car   is: 56.4 %
Accuracy for class bird  is: 0.0 %
Accuracy for class cat   is: 1.5 %
Accuracy for class deer  is: 66.8 %
Accuracy for class dog   is: 28.0 %
Accuracy for class frog  is: 28.4 %
Accuracy for class horse is: 41.3 %
Accuracy for class ship  is: 54.4 %
Accuracy for class truck is: 17.8 %


### **Q 1.5 b)**

In [ ]:
from torchvision import datasets, models, transforms
import torch.optim as optim
import torch
import torch.nn as nn


model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
num_ftrs1 = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs1, 10)
model_conv = model_conv.to(device)

criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

model_conv = train_model(model_conv, criterion1, optimizer1, 3)

Epoch 0/2
----------
[1,  2000] loss: 2.528
[1,  4000] loss: 2.580
[1,  6000] loss: 2.644
[1,  8000] loss: 2.628
[1, 10000] loss: 2.599
[1, 12000] loss: 2.660
Epoch 1/2
----------
[2,  2000] loss: 2.629
[2,  4000] loss: 2.610
[2,  6000] loss: 2.578
[2,  8000] loss: 2.638
[2, 10000] loss: 2.610
[2, 12000] loss: 2.612
Epoch 2/2
----------
[3,  2000] loss: 2.580
[3,  4000] loss: 2.618
[3,  6000] loss: 2.611
[3,  8000] loss: 2.612
[3, 10000] loss: 2.607
[3, 12000] loss: 2.584
Training complete in 4m 32s


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_conv(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 24 %


In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
model_conv.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_conv(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 47.7 %
Accuracy for class car   is: 24.5 %
Accuracy for class bird  is: 50.2 %
Accuracy for class cat   is: 20.1 %
Accuracy for class deer  is: 7.9 %
Accuracy for class dog   is: 10.6 %
Accuracy for class frog  is: 27.4 %
Accuracy for class horse is: 38.8 %
Accuracy for class ship  is: 16.9 %
Accuracy for class truck is: 19.4 %
